- change fold to 5
- modify best_val_loss mistake
- introduce weight decay
- bcewithlogitloss instead of bceloss

In [1]:
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from category_encoders import CountEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss
from sklearn import preprocessing
from tqdm import tqdm_notebook as tqdm
from sklearn.multioutput import MultiOutputClassifier

sys.path.append('../input/multilabelstraifier/')
from ml_stratifiers import MultilabelStratifiedKFold
warnings.filterwarnings('ignore')

import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf

In [2]:
DATA_DIR = '/kaggle/input/lish-moa/'
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
#non_targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
test = pd.read_csv(DATA_DIR + 'test_features.csv')
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [3]:
target_feats = [ i for i in targets.columns if i != "sig_id"]
g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]

In [4]:
noncons_train_index = train[train.cp_type=="ctl_vehicle"].index
cons_train_index = train[train.cp_type!="ctl_vehicle"].index
noncons_test_index = test[test.cp_type=="ctl_vehicle"].index
cons_test_index = test[test.cp_type!="ctl_vehicle"].index

# preprocess

In [5]:
# normalization by ctl group
train_ctl = train[train.index.isin(noncons_train_index)].copy().reset_index(drop=True)
test_ctl = test[test.index.isin(noncons_test_index)].copy().reset_index(drop=True)
ctl_df = pd.concat([train_ctl, test_ctl])

ctl_group_data = ctl_df.groupby(["cp_dose", "cp_time"]).agg({"mean"}).reset_index()
mean_g_feats = ["mean-" + i for i in g_feats]
mean_c_feats = ["mean-" + i for i in c_feats]
columns = ["cp_dose", "cp_time"] + mean_g_feats + mean_c_feats
ctl_group_data.columns = columns

train = train[train.index.isin(cons_train_index)].copy().reset_index(drop=True)
targets = targets[targets.index.isin(cons_train_index)].copy().reset_index(drop=True)
#non_targets = non_targets[non_targets.index.isin(cons_train_index)].copy().reset_index(drop=True)

# Feature engineering 

In [6]:
def fe(df, remove_features):
    df = pd.merge(df, ctl_group_data, on=["cp_time", "cp_dose"], how="left")
    for i in range(len(g_feats)):
        df["diff-g-"+str(i)] = df["g-"+str(i)] - df["mean-g-"+str(i)]
    for i in range(len(c_feats)):
        df["diff-c-"+str(i)] = df["c-"+str(i)] - df["mean-c-"+str(i)]
    
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
        
    df.drop(remove_features, axis=1, inplace=True)
    return df

remove_features = ["cp_type" , "sig_id"] + mean_g_feats + mean_c_feats + g_feats + c_feats 

train = fe(train, remove_features)
test = fe(test, remove_features)

print(train.shape, test.shape)

(21948, 874) (3982, 874)


# modelling

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
batch_size = 128
train_epochs = 40
n_folds=5
EARLY_STOPPING_STEPS = 10

def mean_log_loss(y_true, y_pred):
    metrics = []
    for i, target in enumerate(target_feats):
        metrics.append(log_loss(y_true[:, i], y_pred[:, i].astype(float), labels=[0,1]))
    return np.mean(metrics)

def seed_everything(seed=1234): 
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

class MoaModel(nn.Module):
    def __init__(self, num_columns):
        super(MoaModel, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_columns)
        self.dropout1 = nn.Dropout(0.3)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_columns, 2048))
        
        self.batch_norm2 = nn.BatchNorm1d(2048)
        self.dropout2 = nn.Dropout(0.6)
        self.dense2 = nn.utils.weight_norm(nn.Linear(2048, 1048))
        
        self.batch_norm3 = nn.BatchNorm1d(1048)
        self.dropout3 = nn.Dropout(0.6)
        self.dense3 = nn.utils.weight_norm(nn.Linear(1048, 206))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x
    
def modelling_torch(tr, target, te, sample_seed, init_num):
    seed_everything(seed=sample_seed) 
    X_train = tr.copy()
    y_train = target.copy()
    X_test = te.copy()
    test_len = X_test.shape[0]

    mskf=MultilabelStratifiedKFold(n_splits = n_folds, shuffle=True, random_state=2)
    models = []
    
    X_test = torch.tensor(X_test, dtype=torch.float32)
    X_test = torch.utils.data.TensorDataset(X_test) 
    test_loader = torch.utils.data.DataLoader(X_test, batch_size=batch_size, shuffle=False)
    
    oof = np.zeros([len(X_train),y_train.shape[1]])
    oof_targets = np.zeros([len(X_train),y_train.shape[1]])
    pred_value = np.zeros([test_len, y_train.shape[1]])
    scores = []
    for fold, (train_index, valid_index) in enumerate(mskf.split(X_train, y_train)):
        print("Fold "+str(fold+1))
        X_train2 = torch.tensor(X_train[train_index,:], dtype=torch.float32)
        y_train2 = torch.tensor(y_train[train_index], dtype=torch.float32)

        X_valid2 = torch.tensor(X_train[valid_index,:], dtype=torch.float32)
        y_valid2 = torch.tensor(y_train[valid_index], dtype=torch.float32)
            
        clf = MoaModel(num_columns=init_num)
        loss_fn = torch.nn.BCEWithLogitsLoss() 
        optimizer = optim.Adam(clf.parameters(), lr = 0.001, weight_decay=1e-5) 
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, eps=1e-4, verbose=True)
        
        train = torch.utils.data.TensorDataset(X_train2, y_train2)
        valid = torch.utils.data.TensorDataset(X_valid2, y_valid2)
        
        clf.to(device)
        
        train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True) 
        valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
        
        best_val_loss = np.inf
        for epoch in range(train_epochs):
            start_time = time.time()
            clf.train()
            avg_loss = 0.
            for x_batch, y_batch in tqdm(train_loader, disable=True):
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                y_pred = clf(x_batch) 
                loss = loss_fn(y_pred, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                avg_loss += loss.item() / len(train_loader)        
            
            clf.eval()
            avg_val_loss = 0.
            for i, (x_batch, y_batch) in enumerate(valid_loader): 
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                y_pred = clf(x_batch).detach()
                avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
        
            elapsed_time = time.time() - start_time 
            scheduler.step(avg_val_loss)
            #print('Epoch-{0} lr: {1}'.format(epoch, optimizer.param_groups[0]['lr']))
                    
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                print('Best model: Epoch {} \t loss={:.6f} \t val_loss={:.6f} \t time={:.2f}s'.format(
                    epoch + 1, avg_loss, avg_val_loss, elapsed_time))
                torch.save(clf.state_dict(), 'best-model-parameters.pt')
     
        pred_model = MoaModel(num_columns=init_num)
        pred_model.load_state_dict(torch.load('best-model-parameters.pt'))
        pred_model.eval()
        
        # validation check ----------------
        oof_epoch = np.zeros([X_valid2.size(0), y_train.shape[1]])
        target_epoch = np.zeros([X_valid2.size(0), y_train.shape[1]])
        for i, (x_batch, y_batch) in enumerate(valid_loader): 
                y_pred = pred_model(x_batch).sigmoid().detach()
                oof_epoch[i * batch_size:(i+1) * batch_size,:] = y_pred.cpu().numpy()
                target_epoch[i * batch_size:(i+1) * batch_size,:] = y_batch.cpu().numpy()
        #print("Fold {} log loss: {}".format(fold+1, mean_log_loss(target_epoch, oof_epoch)))
        scores.append(mean_log_loss(target_epoch, oof_epoch))
        oof[valid_index,:] = oof_epoch
        oof_targets[valid_index,:] = target_epoch
        #-----------------------------------
        
        # test predcition --------------
        test_preds = np.zeros([test_len, y_train.shape[1]])
        for i, (x_batch,) in enumerate(test_loader): 
            y_pred = pred_model(x_batch).sigmoid().detach()
            test_preds[i * batch_size:(i+1) * batch_size, :] = y_pred.cpu().numpy()
        pred_value += test_preds / n_folds
        # ------------------------------
        
    for i, ele in enumerate(scores):
        print("Fold {} log loss: {}".format(i+1, scores[i]))
    print("Std of log loss: {}".format(np.std(scores)))
    print("Total log loss: {}".format(mean_log_loss(oof_targets, oof)))
    
    return oof, oof_targets, pred_value

cuda


In [8]:
seeds = [0,1,2,3,4,5,6] 
nn_train = train.copy().to_numpy()
nn_targets = targets.drop("sig_id", axis=1).copy().to_numpy()
nn_test = test.copy().to_numpy()

oof_final = np.zeros([len(train),nn_targets.shape[1]])
pred_final = np.zeros([len(test),nn_targets.shape[1]])

for seed_ in seeds:
    oof, oof_targets, pytorch_pred = modelling_torch(nn_train, nn_targets, nn_test, sample_seed = seed_, init_num = nn_train.shape[1])
    oof_final += oof / len(seeds)
    pred_final += pytorch_pred / len(seeds)
print("Total log loss: {}".format(mean_log_loss(oof_targets, oof_final)))

Fold 1
Best model: Epoch 1 	 loss=0.415437 	 val_loss=0.079371 	 time=1.53s
Best model: Epoch 2 	 loss=0.048834 	 val_loss=0.028417 	 time=0.91s
Best model: Epoch 3 	 loss=0.027082 	 val_loss=0.022981 	 time=0.79s
Best model: Epoch 4 	 loss=0.023576 	 val_loss=0.021647 	 time=0.80s
Best model: Epoch 5 	 loss=0.021531 	 val_loss=0.020388 	 time=0.78s
Best model: Epoch 6 	 loss=0.020788 	 val_loss=0.019412 	 time=0.78s
Best model: Epoch 7 	 loss=0.019910 	 val_loss=0.018859 	 time=0.81s
Best model: Epoch 8 	 loss=0.019516 	 val_loss=0.018564 	 time=0.82s
Best model: Epoch 9 	 loss=0.019460 	 val_loss=0.018366 	 time=0.84s
Best model: Epoch 11 	 loss=0.018790 	 val_loss=0.017921 	 time=0.96s
Best model: Epoch 12 	 loss=0.018150 	 val_loss=0.017586 	 time=0.83s
Best model: Epoch 13 	 loss=0.017815 	 val_loss=0.017411 	 time=0.86s
Best model: Epoch 15 	 loss=0.017509 	 val_loss=0.017212 	 time=0.79s
Best model: Epoch 16 	 loss=0.017328 	 val_loss=0.017035 	 time=0.79s
Best model: Epoch 17 	

In [9]:
#local = targets.drop("sig_id", axis=1).copy()
#local[target_feats] = oof_final
#local.loc[noncons_train_index,target_feats] = 0
#print("Local log loss: {}".format(mean_log_loss(oof_targets, oof_final)))

In [10]:
sub[target_feats] = pred_final
sub.loc[noncons_test_index,target_feats] = 0
sub.to_csv('submission.csv', index=False)